# Load modules

In [1]:
import xarray as xr
#import numpy as np


# Define functions

In [2]:
def derive_n(ds_T): # Derive atmospheric number density - particles cm^-3 (take care with units!)

    ds_n = xr.Dataset()
    
    # Conversion constants
    k = 1.380649e-23            # Boltzmann constant J K^-1 = kg m^2 s^-2 K^-1
    to_cm3 = 1e6                # convertion from m^3 to cm^3
    to_Pa = 100                 # convert from hPa to Pa. Pa = kg m^-1 s^-2
    
    ds_n['n'] = ds_T['lev']*to_Pa/(to_cm3*k*ds_T) 
    # Units: kg m^-1 s^-2 * kg^-1 m^-2 s^2 K * K^-1 = m^-3 | m^-3 * 1e-6 = cm^-3
    
    return ds_n

# Define options

In [6]:

path_in = '../data/processed_data/monthly_domain_average/'      # Path to data on_lev
path_out = '../data/processed_data/monthly_domain_average/'     # Output path to save data to - shouldn't be the same as input, or that will overwrite files, I assume?
file_type = 'nonrr'                                     # 'nonrr' or 'rr'
location = 'cm'                                         # 'cm' for CONUS
time_region = 'night'                                   # 'night' or 'diur'

variables_for_n = ['O','O3']                            # diurnal:['O','NO','CO2','CO','H2O'] # Night:['O','O3']


#Quick sense check of time region of data:

for var in variables_for_n:
    if time_region == 'night' and var in ['NO','CO2','CO','H2O']:
        print(f'{var} requires diurnal averaging!')
    elif time_region == 'diur' and var == 'O3':
        print(f'{var} should use night-time averaging!')

# Convert to number density

In [7]:
# Read in file(s) to ds_on_lev

filestr= f'{file_type}_{time_region}_{location}'
ds_on_lev = xr.open_mfdataset(path_in+ f'{filestr}*')


# Calculate var_n and add into ds_on_lev

for var in variables_for_n:
    ds_on_lev[f'{var}_n'] = ds_in[var]*derive_n(ds_in['T'])['n']
    
# Save dataset out

ds_on_lev.to_netcdf(f'{path_out}{filestr}_2010.nc')
ds_on_lev.close()

OSError: no files to open